In [4]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
import os, sys, glob, warnings, re, math, patsy, csv
import pandas as pd
import numpy as np
from pathlib import Path
import info, utils, preprocess


/anaconda3/envs/social_navigation_analysis/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/anaconda3/envs/social_navigation_analysis/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


# Log parsing dev

In [7]:
file_path = '/Users/matty_gee/Desktop/SNT_data/SNT-fmri_CUD/Data/SNT/Logs/SocialSpace_01-17-23-1_buttonbox_v3.log'
experimenter = 'kb'

In [12]:
experimenter = experimenter.lower()
if experimenter == 'rt': 
    keys = ['30','31'] # 1,2
    tr_key = '63'
elif experimenter in ['af','rr']: 
    keys = ['28','29']
elif experimenter in ['nr','cs','kb','ff']: 
    keys = ['29','30']
    tr_key = '54'

# Read input data into data variable - a list of all the rows in input file
# Each data row has 4 or 8 items, for example:
    #['432843', '[1]', ':', 'Key', '54', 'DOWN', 'at', '418280  ']
    #['384919', '[3986]', ':', 'slide_28_end: 384919']
with open(file_path, 'r') as csvfile: 
    data = [row for row in csv.reader(csvfile, delimiter = '\t') if len(row) == 8 or len(row) == 4]

#------------------------------------------------------------
# parse data into a standardized xlsx
#------------------------------------------------------------

choice_data = pd.DataFrame()

# find the first slide onset --> eg, ['50821', '[11]', ':', 'pic_1_start: 50811']
first_img  = [row for row in data if row[3].startswith('pic_1_start')][0] # the first time the first character's image is shown
task_start = int(first_img[3].split()[1])

for t, trial in info.decision_trials.iterrows():

    slide_num = trial['cogent_slide_num']
    row_num = -1
    press_found = 0 # record if valid button push is found

    # find the row numbdr for this trial 
    while row_num < len(data):
        row_num += 1
        if data[row_num][3].startswith("%s_start" % slide_num):
            break

    # slide start onset
    if row_num < len(data):
        slide_start = int(data[row_num][3].split()[1])
        slide_onset = (slide_start - task_start) / 1000
        slide_end   = slide_start + 11988 # slide end is 11988ms after start
    else:
        print('ERROR: %s_start not found!' % trial['cogent_slide_num'])

    # find choices: find the next 'Key DOWN' slide(s), with a valid press & normal RT
    while row_num < len(data):
        
        row_num += 1

        if (data[row_num][3] == 'Key' and data[row_num][5] == 'DOWN') and \
            (data[row_num][4] == keys[0] or data[row_num][4] == keys[1]):

            # check if rt is within response window
            press_time = int(data[row_num][7])
            if (press_time > slide_start) and (press_time < slide_end): 

                press_found += 1
                key = data[row_num][4]
                rt  = press_time - int(slide_start)
                bp  = (1 if key == keys[0] else 2) 

                # check for conflicts when multiple button presses
                if press_found > 1: # 2nd or more response
                    dec_ = int(trial['cogent_opt1'] if bp == 1 else trial['cogent_opt2'])
                    if verbose: print('conflict: %s: %s, %s' % (slide_num, dec, dec_))
                    if dec_ != dec: press_conflict = 1 
                    else:           press_conflict = 0
                else: # first response (default choice)
                    dec = int(trial['cogent_opt1'] if bp == 1 else trial['cogent_opt2'])
                    press_conflict = np.nan

                if verbose: print('%s: %s, %s, %s, %s' % (slide_num, bp, dec, rt, press_conflict))

        # if slide ends before finding valid button push
        elif "_end" in data[row_num][3]: 
            if press_found == 0: rt, bp, dec, press_conflict  = 0, 0, 0, np.nan
            break
    
    #------------------------------------------------------------
    # output trial info
    #------------------------------------------------------------

    if trial['dimension'] == 'affil': dim_decs = [dec, 0]
    else:                             dim_decs = [0, dec]
    choice_data.loc[t, ['slide_num','decision_num','onset','presses_found','presses_conflict', \
                        'button_press','decision','affil','power','reaction_time']] \
                        = [slide_num, t+1, slide_onset, press_found, press_conflict, bp, dec] + dim_decs + [rt/1000]

slide_4: 1, 1, 2510, nan
slide_6: 2, -1, 724, nan
slide_8: 1, 1, 591, nan
slide_12: 1, 1, 5093, nan
slide_15: 1, 1, 797, nan
slide_18: 2, -1, 1406, nan
slide_21: 2, -1, 4695, nan
slide_22: 2, -1, 1690, nan
slide_26: 1, -1, 606, nan
slide_29: 1, -1, 526, nan
slide_38: 1, 1, 1519, nan
slide_44: 1, 1, 535, nan
slide_48: 1, 1, 4067, nan
slide_51: 1, 1, 4101, nan
slide_54: 1, 0, 865, nan
slide_57: 1, 0, 6639, nan
slide_63: 2, -1, 2400, nan
slide_66: 2, -1, 8670, nan
slide_71: 1, -1, 3058, nan
slide_80: 1, 1, 855, nan
slide_82: 2, 1, 1247, nan
conflict: slide_82: 1, -1
slide_82: 1, 1, 1247, 1
slide_85: 1, 1, 8155, nan
slide_89: 1, -1, 6491, nan
slide_91: 2, -1, 2088, nan
slide_94: 1, 1, 867, nan
slide_96: 1, -1, 5064, nan
slide_101: 1, 1, 5838, nan
slide_106: 2, -1, 731, nan
slide_111: 1, 1, 705, nan
slide_123: 2, 1, 2194, nan
slide_127: 1, -1, 2906, nan
slide_130: 1, -1, 715, nan
slide_133: 2, -1, 564, nan
slide_135: 1, 1, 811, nan
slide_139: 2, -1, 1103, nan
slide_142: 2, 0, 1022, nan
slid

In [6]:
choice_data

,slide_num,decision_num,onset,presses_found,presses_conflict,button_press,decision,affil,power,reaction_time
0,slide_4,1.0,55.905,1.0,NaN,1.0,-1.0,-1.0,0.0,2.510
1,slide_6,2.0,71.887,1.0,NaN,2.0,1.0,1.0,0.0,0.724
2,slide_8,3.0,91.866,1.0,NaN,1.0,-1.0,-1.0,0.0,0.591
3,slide_12,4.0,119.821,1.0,NaN,1.0,-1.0,0.0,-1.0,5.093
4,slide_15,5.0,139.788,1.0,NaN,1.0,-1.0,-1.0,0.0,0.797
...,...,...,...,...,...,...,...,...,...,...
58,slide_203,59.0,1417.696,1.0,NaN,1.0,-1.0,0.0,-1.0,1.517
59,slide_206,60.0,1439.663,1.0,NaN,1.0,1.0,0.0,1.0,0.856
60,slide_208,61.0,1459.642,1.0,NaN,1.0,1.0,0.0,1.0,0.893
61,slide_211,62.0,1479.609,1.0,NaN,1.0,1.0,0.0,1.0,1.027
